In [1]:
%%bash

# mkdir ~/.qlib
# mkdir ~/.qlib/qlib_data
# mkdir ~/.qlib/qlib_data/us_data
# mkdir ~/.qlib/qlib_data/us_data/calendars
# mkdir ~/.qlib/qlib_data/us_data/instruments
# mkdir ~/.qlib/qlib_data/us_data/us_1d_norm

In [1]:
import pandas as pd

In [2]:
import wk_db

wk_db.read_sql('select count(*) from us_market_daily', db_loc='low_freq_db')

,count(*)
0,32616023


In [4]:
dt = wk_db.read_sql('SELECT DISTINCT trade_dt FROM us_market_daily ORDER BY trade_dt;', db_loc='low_freq_db')

In [5]:
dt.to_csv('~/.qlib/qlib_data/us_data/calendars/day.txt', index=False, header=False)

In [6]:
symbol_wk = wk_db.read_sql('SELECT DISTINCT ticker FROM us_market_daily ORDER BY ticker;', db_loc='low_freq_db')

In [7]:
symbol_wk.head()

,ticker
0,A
1,AA
2,AAA
3,AAAU
4,AABA


In [8]:
wk_db.read_sql("SELECT * FROM us_market_daily ORDER BY trade_dt LIMIT 10;", db_loc='low_freq_db')

,trade_dt,wk_inst,ticker,pre_close,open_price,high_price,low_price,close_price,volume,amount,adj_factor,suspension,ext_status
0,1990-01-02,I100000000011260,C,None,137.289157,141.506024,136.084337,141.506024,82478.926000,1.147252e+07,1.0,0,0
1,1990-01-02,I100000000010516,ADM,None,23.112671,23.112671,22.627790,23.004920,677100.193956,1.554928e+07,1.0,0,0
2,1990-01-02,I100000000010800,EACO,None,0.000000,59.375000,53.125000,54.687500,760.000000,4.235480e+04,NaN,0,0
3,1990-01-02,I100000000010901,ITIC,None,5.750000,5.750000,5.750000,5.750000,1000.000000,5.750000e+03,1.0,0,0
4,1990-01-02,I100000000010501,AMWD,None,0.000000,7.000000,6.750000,6.875000,6900.000000,4.751340e+04,NaN,0,0
5,1990-01-02,I100000000010463,REFR,None,0.000000,3.875001,3.500001,3.500001,23300.266667,8.431784e+04,NaN,0,0
6,1990-01-02,I100000000012933,MEI,None,1.508380,1.541900,1.474860,1.508380,140589.500000,2.120624e+05,1.0,0,0
7,1990-01-02,I100000000013777,AMSWA,None,22.500000,22.620000,22.125000,22.500000,73799.333333,1.655780e+06,1.0,0,0
8,1990-01-02,I100000000011466,FACO,None,5.880000,6.000000,5.750000,5.880000,41200.000000,2.421530e+05,1.0,0,0
9,1990-01-02,I100000000011790,ALCO,None,30.250000,30.250000,29.250000,29.250000,18200.000000,5.414500e+05,1.0,0,0


For WK: norm_price = raw_price * adj_factor / (ori_raw_price * ori_adj_factor)

In QLIB: norm_price = raw_price * factor

In [9]:
import pandas as pd

for _, row in symbol_wk.iterrows():
    s = row['ticker']
    print(s)
    # df = wk_db.read_sql("SELECT trade_dt, windcode, open_price, high_price, low_price, close_price, volume, amount, adj_factor FROM a_share_market WHERE windcode='{s}';".format(s=s), db_loc='low_freq_db')
    df = wk_db.read_sql("SELECT * FROM us_market_daily WHERE ticker='{s}' ORDER BY trade_dt;".format(s=s), db_loc='low_freq_db')
    ori_raw_price = df.iloc[0]['close_price']
    ori_adj_factor = df.iloc[0]['adj_factor']
    _df = pd.DataFrame()
    _df['date'] = df['trade_dt']
    _df['symbol'] = df['ticker']
    try:
        _df['factor'] = df['adj_factor'] / (ori_raw_price*ori_adj_factor)
    except:
        _df['factor'] = 1.0
    _df['open'] = df['open_price'] * _df['factor']
    _df['open'] = df['open_price'] *  _df['factor']
    _df['high'] = df['high_price'] * _df['factor']   
    _df['low'] = df['low_price'] * _df['factor']
    _df['close'] = df['close_price'] *  _df['factor']
    _df['volume'] = df['volume']
    _df['amount'] = df['amount'] *  _df['factor']
    # print(df)
    # print(_df)
    wk_symbol = s
    _df.to_csv(f'~/.qlib/qlib_data/us_data/us_1d_norm/{wk_symbol}.csv', index=False)
    # break

A
AA
AAA
AAAU
AABA
AAC
AACG
AACI
AACT
AADI
AADR
AAGR
AAIC
AAL
AAMC
AAME
AAN
AAOI
AAON
AAP
AAPB
AAPD
AAPL
AAPU
AAPY
AAQC
AASP
AAT
AATC
AAWW
AAXJ
AB
ABAT
ABAX
ABB
ABBV
ABC
ABCB
ABCD
ABCL
ABCM
ABCO
ABCS
ABCW
ABDC
ABE
ABEO
ABEQ
ABEV
ABG
ABGI
ABI
ABIL
ABIO
ABL
ABLE
ABLV
ABM
ABMC
ABMD
ABN
ABNB
ABOS
ABP
ABR
ABSI
ABST
ABT
ABTS
ABTX
ABUS
ABVC
ABVX
AC
ACA
ACAB
ACAC
ACAD
ACAH
ACAI
ACAM
ACAQ
ACB
ACBA
ACBI
ACC
ACCD
ACCO
ACDC
ACDI
ACEL
ACER
ACES
ACET
ACEV
ACF
ACFC
ACFN
ACGL
ACGN
ACH
ACHC
ACHL
ACHR
ACHV
ACI
ACIA
ACIC
ACII
ACIO
ACIU
ACIW
ACLS
ACLX
ACM
ACMR
ACMTA
ACN
ACNB
ACNT
ACON
ACP
ACQR
ACR
ACRE
ACRO
ACRS
ACRU
ACRV
ACSI
ACST
ACT
ACTA
ACTC
ACTD
ACTG
ACTT
ACTV
ACTX
ACU
ACUR
ACUS
ACV
ACVA
ACVF
ACW
ACWI
ACWV
ACWX
ACXP
ACY
ADAG
ADAL
ADAP
ADBE
ADC
ADCT
ADD
ADEA
ADER
ADEX
ADFI
ADGE
ADGO
ADI
ADIL
ADIV
ADLI
ADM
ADMA
ADME
ADMS
ADMT
ADN
ADNT
ADP
ADPT
ADPV
ADRA
ADRD
ADRE
ADRO
ADRT
ADRU
ADSE
ADSK
ADST
ADSW
ADT
ADTC
ADTH
ADTN
ADTX
ADUS
ADV
ADVE
ADVM
ADVT
ADX
ADXN
ADXS
ADYX
AE
AEAC
AEAE
AEE
AEF
AEG
AEGN
AEHL
AEHR

In [10]:
all_code = wk_db.read_sql('SELECT ticker, MIN(trade_dt) as st, MAX(trade_dt) as ed FROM us_market_daily GROUP BY ticker ORDER BY ticker;', db_loc='low_freq_db')
all_code

,ticker,st,ed
0,A,1999-11-18,2024-06-26
1,AA,1990-01-02,2024-06-26
2,AAA,2020-09-09,2024-06-26
3,AAAU,2018-08-15,2024-06-26
4,AABA,2017-06-19,2019-10-02
...,...,...,...
12991,ZY,2021-04-22,2022-10-18
12992,ZYME,2017-04-28,2024-06-26
12993,ZYNE,2015-08-05,2023-10-10
12994,ZYXI,2019-02-12,2024-06-26


In [11]:
all_code.to_csv(f'~/.qlib/qlib_data/us_data/instruments/all.txt', index=False, header=False, sep='\t')

In [3]:
idx_map = {
    'spx': "SPX.GI",
    'ndx': "NDX.GI"
}

In [25]:
idx_name = 'spx'
idx_code = idx_map[idx_name]

idx_df = wk_db.read_sql(f"SELECT con_symbol, MIN(trade_dt) as st, MAX(trade_dt) as ed FROM ame_index_components WHERE index_windcode='{idx_code}' GROUP BY con_symbol ORDER BY con_symbol;", db_loc='low_freq_db')
idx_df

,con_symbol,st,ed
0,A,1999-01-04,2024-06-13
1,AA,1999-01-04,2016-10-31
2,AAL,2015-03-23,2024-06-13
3,AAP,2015-07-08,2023-08-24
4,AAPL,1999-01-04,2024-06-13
...,...,...,...
804,YUM,1999-01-04,2024-06-13
805,ZBH,1999-01-04,2024-06-13
806,ZBRA,2019-12-23,2024-06-13
807,ZION,1999-01-04,2024-03-15


In [26]:
idx_df.to_csv(f'~/.qlib/qlib_data/us_data/instruments/{idx_name}.txt', index=False, header=False, sep='\t')

In [27]:
pos_df = pd.read_csv('/home/linq/finance/data/us_position_ref.csv')
pos_df.columns = ['trade_dt', 'dir', 'ticker', 'name', 'weight']
pos_df['ticker'] = pos_df['ticker'].apply(lambda s: s.split('.')[0])
pos_df['trade_dt'] = pd.to_datetime(pos_df['trade_dt'], format='%Y%m%d')
pos_df = pos_df.set_index(['trade_dt', 'ticker'])
pos_df['dir'] = pos_df['dir'].apply(lambda s: 1 if s=='多头' else -1)
pos_df['weight'] = pos_df['dir'] * pos_df['weight']
pos_df

dir                         name  weight
trade_dt   ticker                                          
2024-01-05 ADBE      1                   奥多比(ADOBE)  0.0133
           AJG       1  亚瑟加拉格尔(ARTHUR J. GALLAGHER)  0.0073
           ANET      1                     ARISTA网络  0.0100
           BIIB      1                 生物基因(BIOGEN)  0.0028
           BURL      1     伯灵顿百货(BURLINGTON STORES)  0.0129
...                ...                          ...     ...
2024-06-28 VAL       1                      VALARIS  0.0025
           VRT       1                       VERTIV  0.0084
           VRTX      1                 福泰制药(VERTEX)  0.0041
           VST       1                       VISTRA  0.0054
           WM        1     美国废物管理(WASTE MANAGEMENT)  0.0104

[1813 rows x 3 columns]

In [28]:
idx_df['con_symbol'].values

array(['A', 'AA', 'AAL', 'AAP', 'AAPL', 'ABBV', 'ABK', 'ABMD', 'ABNB',
       'ABS', 'ABT', 'ACAS', 'ACE', 'ACGL', 'ACN', 'ADBE', 'ADI', 'ADM',
       'ADP', 'ADS', 'ADSK', 'ADT', 'AEE', 'AEP', 'AES', 'AET', 'AFL',
       'AGN', 'AIG', 'AIV', 'AIZ', 'AJG', 'AKAM', 'AKS', 'ALB', 'ALGN',
       'ALK', 'ALL', 'ALLE', 'ALTR', 'ALXN', 'AMAT', 'AMCR', 'AMD', 'AME',
       'AMG', 'AMGN', 'AMP', 'AMT', 'AMZN', 'AN', 'ANDV', 'ANET', 'ANF',
       'ANR', 'ANSS', 'AON', 'AOS', 'APA', 'APC', 'APD', 'APH', 'APOL',
       'APTV', 'ARE', 'ARG', 'ARNC', 'ATI', 'ATO', 'ATVI', 'AV', 'AVB',
       'AVGO', 'AVP', 'AVY', 'AWK', 'AXON', 'AXP', 'AYE', 'AYI', 'AZO',
       'BA', 'BAC', 'BALL', 'BAX', 'BBBY', 'BBWI', 'BBY', 'BC', 'BCR',
       'BDX', 'BEAM', 'BEN', 'BF.B', 'BG', 'BHF', 'BIG', 'BIIB', 'BIO',
       'BJS', 'BK', 'BKNG', 'BKR', 'BLDR', 'BLK', 'BMC', 'BMS', 'BMY',
       'BR', 'BRCM', 'BRK.B', 'BRO', 'BS', 'BSX', 'BTU', 'BWA', 'BX',
       'BXLT', 'BXP', 'C', 'CA', 'CAG', 'CAH', 'CAM', 'CARR', 'CA

In [29]:
for ticker in pos_df.index.get_level_values('ticker').values:
    if ticker not in idx_df['con_symbol'].values:        
        idx_df = idx_df.append({'con_symbol':ticker, 'st':'1900-01-01', 'ed':'2099-01-01'}, ignore_index=True)

idx_df

/tmp/ipykernel_1519401/963509206.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  idx_df = idx_df.append({'con_symbol':ticker, 'st':'1900-01-01', 'ed':'2099-01-01'}, ignore_index=True)
/tmp/ipykernel_1519401/963509206.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  idx_df = idx_df.append({'con_symbol':ticker, 'st':'1900-01-01', 'ed':'2099-01-01'}, ignore_index=True)
/tmp/ipykernel_1519401/963509206.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  idx_df = idx_df.append({'con_symbol':ticker, 'st':'1900-01-01', 'ed':'2099-01-01'}, ignore_index=True)
/tmp/ipykernel_1519401/963509206.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instea

,con_symbol,st,ed
0,A,1999-01-04,2024-06-13
1,AA,1999-01-04,2016-10-31
2,AAL,2015-03-23,2024-06-13
3,AAP,2015-07-08,2023-08-24
4,AAPL,1999-01-04,2024-06-13
...,...,...,...
866,BIL,1900-01-01,2099-01-01
867,SHY,1900-01-01,2099-01-01
868,ESTC,1900-01-01,2099-01-01
869,RHHBY,1900-01-01,2099-01-01


In [30]:
idx_df['st'] = '1900-01-01'
idx_df['ed'] = '2099-01-01'

In [31]:
idx_df.to_csv(f'~/.qlib/qlib_data/us_data/instruments/{idx_name}_ext.txt', index=False, header=False, sep='\t')

python ~/finance/quniverse/LiLab/lilab/qlib/scripts/dump_bin.py dump_all --csv_path ~/.qlib/qlib_data/us_data/us_1d_norm --qlib_dir ~/.qlib/qlib_data/us_data --freq day  --exclude_fields date,symbol